## Example N2 Optimization and MD

See https://wiki.fysik.dtu.dk/ase/tutorials/tutorials.html

### Defining N2 molecule and computing atomization energy with EMT potential

In [32]:
from ase import Atoms
from ase.calculators.emt import EMT

atom = Atoms('N')
atom.calc = EMT()
e_atom = atom.get_potential_energy()

d = 1.1
molecule = Atoms('2N', [(0., 0., 0.), (0., 0., d)])
molecule.calc = EMT()
e_molecule = molecule.get_potential_energy()

e_atomization = e_molecule - 2 * e_atom

print('Nitrogen atom energy: %5.2f eV' % e_atom)
print('Nitrogen molecule energy: %5.2f eV' % e_molecule)
print('Atomization energy: %5.2f eV' % -e_atomization)

Nitrogen atom energy:  5.10 eV
Nitrogen molecule energy:  0.44 eV
Atomization energy:  9.76 eV


### Running geometry optimization with BFGS

In [8]:
from ase.optimize import BFGS
dyn = BFGS(molecule)
dyn.run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 18:24:33        6.035493        5.4516
BFGS:    1 18:24:33        5.582586        5.8694
BFGS:    2 18:24:33        5.096836        6.2701
BFGS:    3 18:24:33        4.580312        6.6353
BFGS:    4 18:24:33        4.036764        6.9411
BFGS:    5 18:24:33        3.472142        7.1563
BFGS:    6 18:24:34        2.895239        7.2406
BFGS:    7 18:24:34        2.318495        7.1427
BFGS:    8 18:24:34        1.758995        6.7973
BFGS:    9 18:24:34        1.239699        6.1222
BFGS:   10 18:24:34        0.790959        5.0142
BFGS:   11 18:24:34        0.452392        3.3440
BFGS:   12 18:24:34        0.275168        0.9508
BFGS:   13 18:24:34        0.263524        0.2435
BFGS:   14 18:24:34        0.262779        0.0125


True

#### Optimization using MDMin algorithm

In [ ]:
from ase.optimize import MDMin
optmd = MDMin(molecule)
optmd.run(fmax=0.0005)

### Running and NVE calculation

In [33]:
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase import units
# Describe the interatomic interactions with the Effective Medium Theory

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(molecule, temp=500)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = VelocityVerlet(molecule, 1 * units.fs)  # 5 fs time step
dyn = VelocityVerlet(molecule, 1 * units.fs, trajectory='md.traj', logfile='md.log')  # 5 fs time step



def printenergy(a):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

In [34]:
printenergy(molecule)
for i in range(20):
    dyn.run(10)
    printenergy(molecule)

Energy per atom: Epot = 0.220eV  Ekin = 1070.580eV (T=8282379K)  Etot = 1070.800eV
Energy per atom: Epot = 5.012eV  Ekin = 1065.825eV (T=8245593K)  Etot = 1070.837eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Energy per atom: Epot = 5.100eV  Ekin = 1065.739eV (T=8244928K)  Etot = 1070.839eV
Ener

###  Running MD with NVT (t=300K)

In [27]:
from ase.md import Langevin
dyn = Langevin(molecule, 5 * units.fs, 300, 0.002)

In [31]:
for i in range(50):
    dyn.run(10)
    printenergy(molecule)

Energy per atom: Epot = 5.100eV  Ekin = 496.940eV (T=3844498K)  Etot = 502.040eV
Energy per atom: Epot = 5.100eV  Ekin = 502.891eV (T=3890539K)  Etot = 507.991eV
Energy per atom: Epot = 5.100eV  Ekin = 513.924eV (T=3975893K)  Etot = 519.024eV
Energy per atom: Epot = 5.100eV  Ekin = 481.890eV (T=3728067K)  Etot = 486.990eV
Energy per atom: Epot = 5.100eV  Ekin = 351.229eV (T=2717231K)  Etot = 356.329eV
Energy per atom: Epot = 5.100eV  Ekin = 410.133eV (T=3172928K)  Etot = 415.233eV
Energy per atom: Epot = 5.100eV  Ekin = 372.715eV (T=2883450K)  Etot = 377.815eV
Energy per atom: Epot = 5.100eV  Ekin = 370.981eV (T=2870039K)  Etot = 376.081eV
Energy per atom: Epot = 5.100eV  Ekin = 376.211eV (T=2910499K)  Etot = 381.311eV
Energy per atom: Epot = 5.100eV  Ekin = 394.036eV (T=3048400K)  Etot = 399.136eV
Energy per atom: Epot = 5.100eV  Ekin = 382.701eV (T=2960710K)  Etot = 387.801eV
Energy per atom: Epot = 5.100eV  Ekin = 432.530eV (T=3346202K)  Etot = 437.630eV
Energy per atom: Epot = 5.10

In [39]:
from ase.io import read, write
ta = read('md.traj',index=':')

In [40]:
len(ta)

201

In [41]:
ta[0].get_potential_energy()

0.44034357303561467

In [46]:
for e in ta[:2]:
    print(e.get_potential_energy())

0.44034357303561467
0.3797337310795117


In [47]:
write('md-n2.extxyz',ta[:2],format='extxyz')